In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.getOrCreate()

In [2]:
df = spark.read.parquet(r'C:\work\python-packages\orca\data\membership.parquet')


In [40]:
df.printSchema()

root
 |-- uniqueID: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- recID: string (nullable = true)
 |-- cost: double (nullable = true)
 |-- dates: string (nullable = true)



In [53]:
from pyspark.sql import Window
windowPartition = Window.partitionBy(col('uniqueID')).orderBy(col('dates').asc())
df.withColumn('rank',row_number().over(windowPartition)).filter(col('rank')==1).count()

1000

In [56]:
from pyspark.sql import DataFrame
from pyspark.sql.column import Column
from typing import List
def dedupsByRank(df:DataFrame,
                 partitionBy:List[str]=List[None],
                 orderBy:List[str]=List[None],
                rankType:Column=row_number())-> DataFrame :
    windowBy = Window.partitionBy(*list(map(lambda col_name:col(col_name),partitionBy)))\
                     .orderBy(*list(map(lambda col_name:col(col_name),orderBy)))
    df = df.withColumn('rank',rankType.over(windowPartition)).filter(col('rank')==1).drop('rank')
    return df
           

In [30]:
from typing import List,Dict,Union
from pyspark.sql import DataFrame
from pyspark.sql.column import Column
from pyspark.sql import Window
from pyspark.sql.functions import *
from pyspark.sql.types import *

def dedupsByRank(
    df: DataFrame,
    partitionCols: List,
    orderCols: Union[List, Dict[str, List]],
    rankType:Column = row_number(),
) -> DataFrame:
    orderCols = (
        {"columns": list(orderCols), "asc": [], "desc": []}
        if isinstance(orderCols, list)
        else orderCols
    )
    _ = (
        lambda col_name: col(col_name).asc()
        if col_name in orderCols["asc"]
        else (col(col_name).desc() if col_name in orderCols["desc"] else col(col_name))
    )
    windowBy = Window.partitionBy(
        *list(map(lambda col_name: col(col_name), partitionCols))
    ).orderBy(*list(map(lambda col_name: _(col_name), orderCols['columns'])))
    return (
        df.withColumn("rank", rankType.over(windowBy))
        .filter(col("rank") == 1)
        .drop("rank")
    )

In [124]:
_ =lambda col_name: col(col_name).asc() if col_name in orderCols['asc'] else (col(col_name).desc() if col_name in orderCols['desc'] else col(col_name))

In [126]:
_('dates')

Column<b'dates ASC NULLS FIRST'>

In [54]:
rankType = row_number()
type(rankType)

pyspark.sql.column.Column

In [17]:
partitionBy =  ['uniqueID','x']

orderCols = {
    'columns' : ['dates','cost'],
    'asc' : ['dates'],
    'desc' : ['cost']
}

#orderCols = ['dates','cost']

if isinstance(orderCols,list):
    orderCols = {'columns':list(orderCols),
                    'asc':[],
                    'desc':[]}

def _(col_name):
    if col_name in orderCols['asc']:
        return col(col_name).asc()
    elif col_name in orderCols['desc']:
        return col(col_name).desc()
    else:
        return col(col_name)
list(map(lambda col_name : _(col_name),orderCols['columns']))

[Column<b'dates ASC NULLS FIRST'>, Column<b'cost DESC NULLS LAST'>]

In [29]:
orderCols = {
    'columns' : ['dates','cost'],
    'asc' : ['dates'],
    'desc' : ['cost']
}
orderCols
#dedupsByRank(df,['uniqueID','cost'],orderCols).show()

{'columns': ['dates', 'cost'], 'asc': ['dates'], 'desc': ['cost']}

In [25]:
df.printSchema()

root
 |-- uniqueID: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- recID: string (nullable = true)
 |-- cost: double (nullable = true)
 |-- dates: string (nullable = true)



In [33]:
dedupsByRank(df,['uniqueID'],['dates','cost']).count()

1000

numpy.datetime64('2020-02-20T12:00:12.345000000')

,0,time
0,1582200012345,2020-02-20 12:00:12.345
1,1582200012346,2020-02-20 12:00:12.346


In [ ]:
df 